<p style="font-size: 24px; font-weight: bold;">Hello there!</p>

<p style="font-size: 16px;">This notebook introduces a super simple way to create a submission file for the competition of <b>"Parkinson's Freezing of Gait Prediction"</b>.</p>

<p style="font-size: 16px;">In this notebook, you will create features by combining 3-dimensional accelerometer data values with metadata about subjects in order to detect FoG (Freezing of Gait) events.</p>

<p style="font-size: 16px;">To predict FoG events (<code>'StartHesitation'</code>, <code>'Turn'</code>, <code>'Walking'</code>) at a given time point <code><b>t</b></code>, you will use the accelerometer data values and subject information at the same time point <code><b>t</b></code>.</p>

<p style="font-size: 16px;">However, since this procedure does not handle temporal information well, it must be necessary to conduct innovative feature engineering to achieve better performance.</p>

<p style="font-size: 16px;">The purpose of publishing this notebook is to demonstrate the rough procedure up to submitting results for the competition using simple code as much as possible.</p>

<p style="font-size: 16px;">I hope that the release of this notebook will contribute even a little to the excitement of the competition.</p>

<p style="font-size: 16px;">Let's enjoy Kaggle together!</p>

<h1>Import Modules</h1>

In [26]:
import os
import tqdm
import glob
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

In [6]:
# parent directory
pdir = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction'

# load meta data

In [7]:
df_tdcs_meta = pd.read_csv(os.path.join(pdir, 'tdcsfog_metadata.csv'))
df_tdcs_meta.head()

,Id,Subject,Visit,Test,Medication
0,003f117e14,4dc2f8,3,2,on
1,009ee11563,f62eec,4,2,on
2,011322847a,231c3b,2,2,on
3,01d0fe7266,231c3b,2,1,off
4,024418ba39,fa8764,19,3,on


In [8]:
df_defog_meta = pd.read_csv(os.path.join(pdir, 'defog_metadata.csv'))
df_defog_meta.head()

,Id,Subject,Visit,Medication
0,02ab235146,e1f62e,2,on
1,02ea782681,ae2d35,2,on
2,06414383cf,8c1f5e,2,off
3,092b4c1819,2874c5,1,off
4,0a900ed8a2,0e3d49,2,on


In [9]:
df_subjects = pd.read_csv(os.path.join(pdir, 'subjects.csv'))
df_subjects.head()

,Subject,Visit,Age,Sex,YearsSinceDx,UPDRSIII_On,UPDRSIII_Off,NFOGQ
0,00f674,2.0,63,M,27.0,43.0,49.0,24
1,00f674,1.0,63,M,27.0,31.0,30.0,26
2,02bc69,NaN,69,M,4.0,21.0,NaN,22
3,040587,2.0,75,M,26.0,52.0,69.0,21
4,040587,1.0,75,M,26.0,47.0,75.0,24


# Load tdcsfog data

In [10]:
# list of all tdcsfog csv file path
tdcs_file_path = glob.glob(os.path.join(pdir, 'train', 'tdcsfog', '*.csv'), recursive=True)

    # In this notebook, we limit the number of files to be read in order to reduce the time required for model training.

tdcs_file_path = tdcs_file_path[::10] #weghalen!!!!!!!!!!!!!

print(f'the number of files to be read: {len(tdcs_file_path)}')

pd.read_csv(tdcs_file_path[0]).head()

the number of files to be read: 84


,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking
0,0,-9.665890,0.042550,0.184744,0,0,0
1,1,-9.672969,0.049217,0.184644,0,0,0
2,2,-9.670260,0.033620,0.193790,0,0,0
3,3,-9.673356,0.035159,0.184369,0,0,0
4,4,-9.671458,0.043913,0.197814,0,0,0


In [11]:
# Initialize a DataFrame to combine data from multiple CSV files.
df_tdcs = pd.DataFrame()

# load tdcsfog time series in combination with metadata.
for fp in tqdm.tqdm(tdcs_file_path):    
    
    # load data into a variable 'tmp'.
    tmp = pd.read_csv(fp)
    
    # get file Id from csv file name.
    file_id = os.path.basename(fp).replace(".csv", "")
    
    # get subject Id.
    subject = df_tdcs_meta.loc[df_tdcs_meta['Id'] == file_id, 'Subject'].iloc[0]
    
    # add metadata.
    tmp['Medication'] = df_tdcs_meta.loc[df_tdcs_meta['Id'] == file_id, 'Medication'].iloc[0]
    tmp['Age'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Age'].iloc[0]
    tmp['Sex'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Sex'].iloc[0]
    tmp['YearsSinceDx'] = df_subjects.loc[df_subjects['Subject'] == subject, 'YearsSinceDx'].iloc[0]
    tmp['NFOGQ'] =df_subjects.loc[df_subjects['Subject'] == subject, 'NFOGQ'].iloc[0]
    
    # concat the data
    df_tdcs = pd.concat([df_tdcs, tmp]).reset_index(drop=True)

100%|██████████| 84/84 [00:08<00:00, 10.04it/s]


In [12]:
# check the contents of the df_tdcs
df_tdcs.head()

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Medication,Age,Sex,YearsSinceDx,NFOGQ
0,0,-9.665890,0.042550,0.184744,0,0,0,off,66,M,9.0,13
1,1,-9.672969,0.049217,0.184644,0,0,0,off,66,M,9.0,13
2,2,-9.670260,0.033620,0.193790,0,0,0,off,66,M,9.0,13
3,3,-9.673356,0.035159,0.184369,0,0,0,off,66,M,9.0,13
4,4,-9.671458,0.043913,0.197814,0,0,0,off,66,M,9.0,13


# Load defog data

In [13]:
# list of all tdcsfog csv file path
defog_file_path = glob.glob(os.path.join(pdir, 'train', 'defog', '*.csv'), recursive=True)

# In this notebook, we limit the number of files to be read in order to reduce the time required for model training.
defog_file_path = defog_file_path[::1]

print(f'the number of files to be read: {len(defog_file_path)}')

the number of files to be read: 91


In [14]:
# Initialize a DataFrame to combine data from multiple CSV files.
df_defog = pd.DataFrame()

for fp in tqdm.tqdm(defog_file_path):
    # load data into a variable 'tmp'.
    tmp = pd.read_csv(fp)
    
    # get file Id from csv file name.
    file_id = os.path.basename(fp).replace(".csv", "")
    
    # get subject Id.
    subject = df_defog_meta.loc[df_defog_meta['Id'] == file_id, 'Subject'].iloc[0]
    
    # add metadata.
    tmp['Medication'] = df_defog_meta.loc[df_defog_meta['Id'] == file_id, 'Medication'].iloc[0]
    tmp['Age'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Age'].iloc[0]
    tmp['Sex'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Sex'].iloc[0]
    tmp['YearsSinceDx'] = df_subjects.loc[df_subjects['Subject'] == subject, 'YearsSinceDx'].iloc[0]
    tmp['NFOGQ'] =df_subjects.loc[df_subjects['Subject'] == subject, 'NFOGQ'].iloc[0]
    
    # extract data from the time period where Valid and Task are both True.
    tmp = tmp[(tmp['Valid'] == True) & (tmp['Task']==True)]
    tmp = tmp.drop(['Valid', 'Task'], axis=1)
    
    # concat the data
    df_defog = pd.concat([df_defog, tmp]).reset_index(drop=True)

100%|██████████| 91/91 [01:37<00:00,  1.07s/it]


In [15]:
# check the contents of the df_defog
df_defog.head()

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Medication,Age,Sex,YearsSinceDx,NFOGQ
0,1000,-1.015138,-0.071969,0.161124,0,0,0,on,72,M,16.0,18
1,1001,-1.018566,-0.057850,0.144988,0,0,0,on,72,M,16.0,18
2,1002,-1.037102,-0.054200,0.129327,0,0,0,on,72,M,16.0,18
3,1003,-1.036416,-0.060578,0.115451,0,0,0,on,72,M,16.0,18
4,1004,-1.012407,-0.069198,0.110121,0,0,0,on,72,M,16.0,18


# Prepare train data

In [102]:
# concat tdcs and defog data.
df_train = pd.concat([df_tdcs, df_defog]).reset_index(drop=True)
# df_train.head(100)
print(len(df_train))
print(len(np.where(df_train['StartHesitation'] == 1)[0]))
print(len(np.where(df_train['Walking'] == 1)[0]))
print(len(np.where(df_train['Turn'] == 1)[0]))

4653288
5281
100280
642441


In [30]:
# encode string columns into 0/1 format
df_train['Medication'] = np.where(df_train['Medication']=='on', 1, 0)
df_train['Sex'] = np.where(df_train['Sex']=='M', 1, 0)
df_train.head()

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Medication,Age,Sex,YearsSinceDx,NFOGQ
0,0,-9.665890,0.042550,0.184744,0,0,0,0,66,1,9.0,13
1,1,-9.672969,0.049217,0.184644,0,0,0,0,66,1,9.0,13
2,2,-9.670260,0.033620,0.193790,0,0,0,0,66,1,9.0,13
3,3,-9.673356,0.035159,0.184369,0,0,0,0,66,1,9.0,13
4,4,-9.671458,0.043913,0.197814,0,0,0,0,66,1,9.0,13


# Split train data for 3 classes

In [46]:
df_train_turn = df_train[df_train['Turn'] == 1]
df_train_walking = df_train[df_train['Walking'] == 1]
df_train_start = df_train[df_train['StartHesitation'] == 1]

In [49]:
# split data into features and target.
y_turn = df_train['Turn']                       # target
X_turn = df_train.drop(['StartHesitation', 'Turn', 'Walking', 'Time'], axis=1)  # feature

y_walking = df_train['Walking']                       # target
X_walking = df_train.drop(['StartHesitation', 'Turn', 'Walking', 'Time'], axis=1)  # feature

y_start = df_train['StartHesitation']                       # target
X_start = df_train.drop(['StartHesitation', 'Turn', 'Walking', 'Time'], axis=1)  # feature

In [68]:
df_train['label'] = np.where(df_train['Turn'] == 1, 1,
                    np.where(df_train['Walking'] == 1, 2,
                    np.where(df_train['StartHesitation'] == 1, 3, 0)))

# print(np.where(df_train['Turn'] + df_train['Walking'] + df_train['StartHesitation'] >1))

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Medication,Age,Sex,YearsSinceDx,NFOGQ,label
0,0,-9.665890,0.042550,0.184744,0,0,0,0,66,1,9.0,13,0
1,1,-9.672969,0.049217,0.184644,0,0,0,0,66,1,9.0,13,0
2,2,-9.670260,0.033620,0.193790,0,0,0,0,66,1,9.0,13,0
3,3,-9.673356,0.035159,0.184369,0,0,0,0,66,1,9.0,13,0
4,4,-9.671458,0.043913,0.197814,0,0,0,0,66,1,9.0,13,0


In [83]:
X_data = df_train.drop(['StartHesitation', 'Turn', 'Walking', 'Time', 'label'], axis=1)  # feature
y_data = df_train['label']

# SMOTE and CV

In [106]:
smote = SMOTE(random_state = 4, k_neighbors=100)

X_syn, y_syn = smote.fit_resample(X_data, y_data)

y_syn.head()
print(len(y_syn))


15621144


In [107]:
print(len(np.where(y_syn == 0)[0]))
print(len(np.where(y_syn == 1)[0]))
print(len(np.where(y_syn == 2)[0]))
print(len(np.where(y_syn == 3)[0]))

3905286
3905286
3905286
3905286


In [119]:
X_syn['StartHesitation'] = 0
X_syn['Turn'] = 0
X_syn['Walking'] = 0
X_syn['StartHesitation'][np.where(y_syn == 3)[0]] = 1
X_syn['Turn'][np.where(y_syn == 1)[0]] = 1
X_syn['Walking'][np.where(y_syn == 2)[0]] = 1
X_syn.head()

/tmp/ipykernel_34/2214224030.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_syn['StartHesitation'][np.where(y_syn == 3)[0]] = 1
/tmp/ipykernel_34/2214224030.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_syn['Turn'][np.where(y_syn == 1)[0]] = 1
/tmp/ipykernel_34/2214224030.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_syn['Walking'][np.where(y_syn == 2)[0]] = 1


,AccV,AccML,AccAP,Medication,Age,Sex,YearsSinceDx,NFOGQ,StartHesitation,Turn,Walking
0,-9.665890,0.042550,0.184744,0,66,1,9.0,13,0,0,0
1,-9.672969,0.049217,0.184644,0,66,1,9.0,13,0,0,0
2,-9.670260,0.033620,0.193790,0,66,1,9.0,13,0,0,0
3,-9.673356,0.035159,0.184369,0,66,1,9.0,13,0,0,0
4,-9.671458,0.043913,0.197814,0,66,1,9.0,13,0,0,0


In [120]:
# split data into features and target.
y = X_syn[['StartHesitation', 'Turn', 'Walking']]                       # target
X = X_syn.drop(['StartHesitation', 'Turn', 'Walking'], axis=1)  # feature

In [55]:
# check the contents of the target
y.head()
print(y.shape)
from collections import Counter

# count the frequency of each class
count = Counter(y)
print(count)

(4653288, 3)
Counter({'StartHesitation': 1, 'Turn': 1, 'Walking': 1})


In [38]:
# check the contents of the feature
X.head()
print(X.shape)

(4653288, 8)


# Train RandomForestClassifier

In [122]:
# train the model with default parameter
rf = RandomForestClassifier(random_state=0)
rf.fit(X, y)

KeyboardInterrupt: 

# Prepare test data

## Process the test data in the same way as the training data

In [ ]:
# list of all tdcsfog csv file path
tdcs_test_file_path = glob.glob(os.path.join(pdir, 'test', 'tdcsfog', '*.csv'), recursive=True)
print(f'the number of files to be read: {len(tdcs_test_file_path)}')

In [ ]:
# Initialize a DataFrame to combine data from multiple CSV files.
df_tdcs_test = pd.DataFrame()

for fp in tqdm.tqdm(tdcs_test_file_path):
    
    # load data into a variable 'tmp'.
    tmp = pd.read_csv(fp)
    
    # get file Id from csv file name.
    file_id = os.path.basename(fp).replace(".csv", "")
    
    # get subject Id.
    subject = df_tdcs_meta.loc[df_tdcs_meta['Id'] == file_id, 'Subject'].iloc[0]
    
    # add metadata.
    tmp['Medication'] = df_tdcs_meta.loc[df_tdcs_meta['Id'] == file_id, 'Medication'].iloc[0]
    tmp['Age'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Age'].iloc[0]
    tmp['Sex'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Sex'].iloc[0]
    tmp['YearsSinceDx'] = df_subjects.loc[df_subjects['Subject'] == subject, 'YearsSinceDx'].iloc[0]
    tmp['NFOGQ'] =df_subjects.loc[df_subjects['Subject'] == subject, 'NFOGQ'].iloc[0]
    
    # add Id data to submit.
    tmp['Id'] = file_id + '_' + tmp['Time'].astype(str)
    
    # concat the data
    df_tdcs_test = pd.concat([df_tdcs_test, tmp]).reset_index(drop=True)

In [ ]:
# check the contents of the df_tdcs_test
df_tdcs_test.head()

In [ ]:
# list of all tdcsfog csv file path
defog_test_file_path = glob.glob(os.path.join(pdir, 'test', 'defog', '*.csv'), recursive=True)
print(f'the number of files to be read: {len(defog_test_file_path)}')

In [ ]:
# Initialize a DataFrame to combine data from multiple CSV files.
df_defog_test = pd.DataFrame()

for fp in tqdm.tqdm(defog_test_file_path):
    # load data into a variable 'tmp'.
    tmp = pd.read_csv(fp)
    
    # get file Id from csv file name.
    file_id = os.path.basename(fp).replace(".csv", "")
    
    # get subject Id.
    subject = df_defog_meta.loc[df_defog_meta['Id'] == file_id, 'Subject'].iloc[0]
    
    # add metadata.
    tmp['Medication'] = df_defog_meta.loc[df_defog_meta['Id'] == file_id, 'Medication'].iloc[0]
    tmp['Age'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Age'].iloc[0]
    tmp['Sex'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Sex'].iloc[0]
    tmp['YearsSinceDx'] = df_subjects.loc[df_subjects['Subject'] == subject, 'YearsSinceDx'].iloc[0]
    tmp['NFOGQ'] =df_subjects.loc[df_subjects['Subject'] == subject, 'NFOGQ'].iloc[0]
    
    # add Id data to submit.
    tmp['Id'] = file_id + '_' + tmp['Time'].astype(str)
    
    # concat the data
    df_defog_test = pd.concat([df_defog_test, tmp]).reset_index(drop=True)

In [ ]:
# check the contents of the df_defog_test
df_defog_test.head()

In [ ]:
# concat tdcs and defog data.
df_test = pd.concat([df_tdcs_test, df_defog_test]).reset_index(drop=True)

# encode string columns into 0/1 format
df_test['Medication'] = np.where(df_test['Medication']=='on', 1, 0)
df_test['Sex'] = np.where(df_test['Sex']=='M', 1, 0)
display(df_test)

In [ ]:
# split data into submission Id and feature.
Id = df_test['Id']                             # Id for submission data
X_test = df_test.drop(['Time', 'Id'], axis=1)  # feature of test data
X_test.head()

# Predict and submit

In [ ]:
# calculate prediction using trained RandomForestClassifier model.
prediction = rf.predict_proba(X_test)

In [ ]:
# Prepare submit data
submit = pd.DataFrame(Id, columns=['Id'])
submit['StartHesitation'] = prediction[:, 0]
submit['Turn'] = prediction[:, 1]
submit['Walking'] = prediction[:, 2]

In [ ]:
display(submit)

In [ ]:
# Save the created submission data.
submit.to_csv('submission.csv', index=False)

In [ ]:
submit

<p style="font-size: 24px; font-weight: bold;">Congratulations!</p>

<p style="font-size: 16px;">You're now ready to submit your work on Kaggle!</p>

<p style="font-size: 16px;">Enjoy your experience on Kaggle!</p>